## Common Modulus Attack

What will happen when the same message is encrypted twice with the same modulus but using a different public key?

In [2]:
import secrets

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

In [4]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = secrets.randbelow(p-2) + 2 

        while gcd(a,p) != 1:
            a = secrets.randbelow(p-2) + 2
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [5]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [6]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

In [7]:
def RSA_generate_d(e, p, q):
    N = p * q
    phiN = (p-1)*(q-1)
    assert gcd(e, phiN) == 1, "GCD of e and Phi N must be 1"
    d = pow(e, -1, phiN)

    return d


In [8]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

In [9]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


## Known to attacker<br>
e1 = public key of first cipher text<br>
c1 = first cipher text<br>
e2 = public key of the second cipher text<br>
c2 = second cipher text<br>
n = modulus that is common to both cipher text

In [40]:
p = generate_prime(512, 3)
q = generate_prime(512, 3)

N = p * q

e1 = 17 
e2 = 257 


In [41]:
d1 = RSA_generate_d(e1, p, q)
d2 = RSA_generate_d(e2, p, q)

message = "Your Auntie very pretty"
ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

89111117114032065117110116105101032118101114121032112114101116116121


226

Encrypting the same message twice using different key same modulus 

In [42]:
encrypted_message1 = RSA_encrypt(ascii_message, e1, N)
encrypted_message2 = RSA_encrypt(ascii_message, e2, N)

### Conditions of the attack

Must make sure that these inverses exist in the first place:
- gcd(e1, e2) = 1
- gcd(c2, n) = 1

In [43]:
while True:
    if gcd(e1, e2) != 1:
        print('pick a new e1 and e2')
        break
    if gcd(encrypted_message2, N) != 1:
        p = generate_prime(512, 3)
        q = generate_prime(512, 3)
        N = p * q
    else:
        break


## Test
print(gcd(e1,e2))
print(gcd(encrypted_message2,N))

1
1


Inverse mod and extended euclid algo 

In [44]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y =egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise ValueError('Modular inverse does not exist.')
    else:
        return x % m

Attack

In [45]:
def attack(encrypted_message1, encrypted_message2, e1, e2, N):
    s1 = modinv(e1,e2)
    s2 = (gcd(e1,e2) - e1 * s1) / e2
    temp = modinv(encrypted_message2, N)
    m1 = pow(encrypted_message1,s1,N)
    m2 = pow(temp,-s2,N)
    return (m1 * m2) % N

In [46]:
message = attack(encrypted_message1, encrypted_message2, e1, e2, N)
print(message)

TypeError: pow() 3rd argument not allowed unless all arguments are integers